In [3]:
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import isnan, when, count, col, lit
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder

data= spark.read.csv("file:/home/hduser/data.csv", inferSchema=True, header=True)

data = data.na.drop()

assembler = VectorAssembler(inputCols=["Year",
                                      "Engine HP",
                                      "Engine Cylinders",
                                      "Number of Doors",
                                      "highway MPG",
                                      "city mpg",
                                      "Popularity"], outputCol = "Attributes")

regressor = RandomForestRegressor(featuresCol = "Attributes", labelCol="MSRP")
pipeline = Pipeline(stages=[assembler, regressor])

pipeline.write().overwrite().save("file:/home/hduser/car_predicton_pipeline")

paramGrid = ParamGridBuilder().addGrid(regressor.numTrees, [100, 500]).build()
pipelineModel = Pipeline.load("file:/home/hduser/car_predicton_pipeline")

In [5]:
crossval = CrossValidator(estimator=pipelineModel,
estimatorParamMaps = paramGrid,
evaluator= RegressionEvaluator(labelCol = "MSRP"),
                               numFolds=3)

In [6]:
train_data, test_data = data.randomSplit([0.8,0.2], seed=123)
cvModel= crossval.fit(train_data)
pred = cvModel.transform(test_data)
pred.select("MSRP", "prediction").show()

+-----+------------------+
| MSRP|        prediction|
+-----+------------------+
|29980|32670.576444206225|
|28030|32696.547383888148|
|30030|32696.547383888148|
|32700| 36208.57084908039|
|29350|23751.960100457116|
|31890| 27657.73916082373|
|34980| 27657.73916082373|
| 2799| 5227.632447783472|
| 2827|  5353.89733100318|
| 3381| 5925.980582254168|
|24450|27081.359913798122|
|21050|23832.820939040375|
| 2000| 5915.912943006891|
| 2181| 7951.718078495673|
| 2144| 5897.823516537988|
| 2265| 7974.922260898618|
|56780| 39464.79021255426|
|49440| 39464.17362299521|
|50640| 39464.17362299521|
|52640| 39464.17362299521|
+-----+------------------+
only showing top 20 rows

